In [2]:
import os
import glob
import pandas as pd
import warnings
import numpy as np
warnings.filterwarnings('ignore')

In [3]:
allFiles = glob.glob(os.path.join("../../data_raw/","*"))

df_logs_full = pd.DataFrame()
for file in allFiles:
    df = pd.read_table(file, delimiter="\n",index_col=None, header=0, error_bad_lines=False, names=['logs'],)
    df_logs_full = pd.concat([df_logs_full, df])

In [10]:
df_logs = df_logs_full[df_logs_full['logs'].str.contains(';Job;', regex=False)]
df_logs = df_logs['logs'].str.split(pat=";", expand=True)
df_logs.columns = ['date','job_id','pbs_status','key','job_name','status']
df_logs = df_logs[df_logs['status'].str.contains('Resource_List|Confirming ALPS reservation|Started, pid|Terminated', regex=True)]
df_logs = df_logs.replace(r'^Resource_List.*','place', regex=True)
df_logs = df_logs.replace(r'^Confirming ALPS reservation.*','confirmed', regex=True)
df_logs = df_logs.replace(r'^Started, pid.*','started', regex=True)
df_logs = df_logs.replace(r'^Terminated.*','terminated', regex=True)
df_logs['date']=pd.to_datetime(df_logs['date'] , format='%m/%d/%Y %H:%M:%S')
df_logs_filtrados = df_logs[['job_name','status','date']]
df_logs_status = df_logs_filtrados.pivot(index='job_name', columns='status', values='date')
df_logs_status['time_allocation'] = (df_logs_status['started'] - df_logs_status['confirmed']).dt.seconds/60
df_logs_status['time_run'] = (df_logs_status['terminated'] - df_logs_status['started']).dt.seconds/60
df_logs_status['run'] = (df_logs_status['terminated'] - df_logs_status['started']).dt.seconds/60
df_logs_status.head(15)

status,confirmed,place,started,terminated,time_allocation,time_run,run
job_name,,,,,,,
1321791.sdb,NaT,NaT,NaT,2021-03-10 01:58:37,NaN,NaN,NaN
1321792.sdb,NaT,NaT,NaT,2021-03-10 01:58:40,NaN,NaN,NaN
1321909.sdb,NaT,NaT,NaT,2021-03-10 00:00:21,NaN,NaN,NaN
1321910.sdb,NaT,NaT,NaT,2021-03-10 00:01:22,NaN,NaN,NaN
1321911.sdb,NaT,NaT,NaT,2021-03-10 00:02:02,NaN,NaN,NaN
1321912.sdb,2021-03-10 00:00:48,2021-03-10 00:00:47,2021-03-10 00:00:48,2021-03-10 00:02:57,0.000000,2.150000,2.150000
1321913.sdb,2021-03-10 00:01:28,2021-03-10 00:01:28,2021-03-10 00:01:29,2021-03-10 00:03:30,0.016667,2.016667,2.016667
1321914.sdb,2021-03-10 00:02:09,2021-03-10 00:02:08,2021-03-10 00:02:09,2021-03-10 00:04:10,0.000000,2.016667,2.016667
1321915.sdb,2021-03-10 00:03:09,2021-03-10 00:03:08,2021-03-10 00:03:10,2021-03-10 00:05:19,0.016667,2.150000,2.150000


In [26]:
df_id_user_name = df_logs_full[df_logs_full['logs'].str.contains(r'<.*user_name', regex=True)]
df_id_user_name = df_id_user_name.replace( r'^.*user_name="|" batch_id="|">$',";", regex=True).replace(r'^;|;$',"", regex=True)
df_id_user_name = df_id_user_name['logs'].str.split(pat=";", expand=True)
df_id_user_arc = df_logs_full[df_logs_full['logs'].str.contains(r'<.*architecture', regex=True)]
df_id_user_arc = df_id_user_arc.replace({'logs': r'^.*<ReserveParam architecture="|" width="|" depth="|" nppn="|">$' }, {'logs': ";"}, regex=True).replace(r'^;|;$',"", regex=True)
df_id_user_arc = df_id_user_arc['logs'].str.split(pat=";", expand=True)
df_id_user_arc.index = df_id_user_arc.index -1
df_id = df_id_user_name.join(df_id_user_arc, lsuffix='_user_name', rsuffix='_arc')
df_id.columns = ['user_name','job_name','architecture','width','depth','nppn']
# Altera o Nome do User para Git
nomes_antivos = df_id['user_name'].unique()
nomes_novos = [f"user{x}" for x in list(range(0,len(nomes_antivos)))]
df_id['user_name'] = df_id['user_name'].replace(nomes_antivos, nomes_novos)
df_id.head()

,user_name,job_name,architecture,width,depth,nppn
9,user0,1334289.sdb,XT,1160,1,40
9,user0,1334289.sdb,XT,1160,1,40
9,user0,1334289.sdb,XT,40,1,40
9,user0,1334289.sdb,XT,1,1,1
9,user0,1331240.sdb,XT,1160,1,40


In [27]:
df_final = df_id.set_index('job_name').join(df_logs_status)
df_final.to_csv('../../data/teste.csv', sep=';', encoding='utf-8')
df_final.head(15)

,user_name,architecture,width,depth,nppn,confirmed,place,started,terminated,time_allocation,time_run,run
job_name,,,,,,,,,,,,
1321912.sdb,user3,XT,2,40,1,2021-03-10 00:00:48,2021-03-10 00:00:47,2021-03-10 00:00:48,2021-03-10 00:02:57,0.000000,2.150000,2.150000
1321913.sdb,user3,XT,1160,1,40,2021-03-10 00:01:28,2021-03-10 00:01:28,2021-03-10 00:01:29,2021-03-10 00:03:30,0.016667,2.016667,2.016667
1321913.sdb,user3,XT,2,40,1,2021-03-10 00:01:28,2021-03-10 00:01:28,2021-03-10 00:01:29,2021-03-10 00:03:30,0.016667,2.016667,2.016667
1321914.sdb,user3,XT,40,1,40,2021-03-10 00:02:09,2021-03-10 00:02:08,2021-03-10 00:02:09,2021-03-10 00:04:10,0.000000,2.016667,2.016667
1321914.sdb,user3,XT,2,40,1,2021-03-10 00:02:09,2021-03-10 00:02:08,2021-03-10 00:02:09,2021-03-10 00:04:10,0.000000,2.016667,2.016667
1321915.sdb,user3,XT,2,40,1,2021-03-10 00:03:09,2021-03-10 00:03:08,2021-03-10 00:03:10,2021-03-10 00:05:19,0.016667,2.150000,2.150000
1321916.sdb,user3,XT,2,40,1,2021-03-10 00:03:50,2021-03-10 00:03:49,2021-03-10 00:03:50,2021-03-10 00:05:49,0.000000,1.983333,1.983333
1321917.sdb,user3,XT,2,40,1,2021-03-10 00:04:50,2021-03-10 00:04:49,2021-03-10 00:04:51,2021-03-10 00:07:03,0.016667,2.200000,2.200000
1321918.sdb,user3,XT,1,40,1,2021-03-10 00:05:31,2021-03-10 00:05:30,2021-03-10 00:05:31,2021-03-10 00:07:34,0.000000,2.050000,2.050000
